In [ ]:
from math import *
from matplotlib.pyplot import *
from IPython.display import HTML
from matplotlib import animation


<div id="haut2page" style="border:3px solid black; padding:3%; font-size:314% ;border-radius : 19px; text-align:center; background-color:#ED6365;font-family: 'Trebuchet MS'">
    SAE MATH 

</div>


# Sujet : Chaine, chemin et circuit eulérien : algorithme de Hierholzer

**Problèmatique** : Damien, un agriculteur étant spécialisé dans les pommes de terre rencontre un problème.
Il possède un petit terrain et ne peut pas se permettre de repasser à pied par une pousse déjà plantée.
Il songe a prendre une machine qui lui permettrait de planter ses pommes de terre sans problème mais avant ça il souhaiterait savoir s'il est possible de planter toute ses pomme de terres sans repasser par une pousse déjà plantée.

Nous allons modèliser le problème à l'aide de graphe.



# I - Génération de la matrice

In [ ]:
# Ici nous avons créé une matrice orientée générée aléatoirement

from random import randint
def ligne(n):
    mat = []
    for i in range(n):
        mat.append(randint(0,1))
    return mat


In [ ]:

def matrice(n):
    Matrice = []
    for i in range(n):
        Matrice.append(ligne(n))
    return Matrice  
        
M = matrice(4)

for li in M:
    print(li)
   

Mais dans notre cas nous devons avoir une matrice non orientée.
Voyons comment faire.
##### Etape n°1 : On enlève les boucles

In [ ]:
j = 0
for i in range(4):
    M[i][j] = 0 #On ajoute des 0 sur la diagonale --> si M[0][0] M[1][1] etc..
    j += 1

for li in M: #Pour avoir un affichage lisible
    print(li)

##### Etape n°2 : Après avoir enlevé les boucles on s'occupe de la symétrie 

In [ ]:
for i in range(4): #Les colonnes 
    for j in range(4): #Les lignes
        if M[j][i] == 1: #Si il y a un 1 au coordonnées M[0][3] on ajoute 1 au coordonées M[3][0]
            M[i][j] = 1
            
            
for li in M:
    print(li)

Voici la fonction `MatriceDeso` qui génère aléatoirement une matrice non orientée 

In [ ]:
#Collage des bouts de code vu ci-dessus

def MatriceDeso(n):
    Ma = matrice(n)
    j = 0
    
    for i in range(n):
        Ma[i][j] = 0
        j += 1
        
    for i in range(n):
        for k in range(n):
            if Ma[k][i] == 1:
                Ma[i][k] = 1
                
    return Ma

#---------Test---------
Mat = MatriceDeso(4)
for li in Mat:
    print(li)
            
    

# II - Circuit eulérien

Pour trouver un circuit eulérien nous devons savoir si :
 - **Le graphe est connexe**.
 - **Il existe un circuit eulérien**.

Pour savoir si le graphe généré est connexe nous allons utiliser le parcours en profondeur pour vérifier si le graphe passe par tout les sommets

Avant de faire la fonction nous allons faire des fonctions qui nous serons utile :
- `voisinDe` qui prend en paramètre un sommet et qui met dans un tableau les voisins du sommets.

- `MatToDico` prend en paramètre un graphe et le converti en dictionnaire avec comme clé les sommets et comme valeur ses voisins.


#####  1. VoisinDe et MatToDico

In [ ]:
# 0 1 2 3 ---> les sommets 
# Par exemple si un 1 est present on l'ajoute dans une table voisins avec son indice.

def voisinDe(li):
    vois = [] #Création de la table
    for i,val in enumerate(li):
        if val == 1:
            vois.append(i) #On ajoute pas la valeur mais l'indice

    return vois

In [ ]:
def MatToDico(G):
    dico = {} #Création du dictionnaire
    for i,li in enumerate(G):
        dico[i] = voisinDe(li) #Liste les voisins par sommet
    return dico

print(MatToDico(Mat))

##### 2. Le parcous en pronfondeur
Nous allons utiliser cette fonction de manière reccursive, c'est à dire que nous allons appeler la fonction dans la fonction.  
Car si nous arrivons sur un sommet dejà visité on retourne en arrière jusqu'à avoir un sommet non visité


In [ ]:
#graphe ----> un dictionnaire; s ----> un sommet; Marq ----> un tableau vide 
def parcours_en_profondeur(graphe, s, Marq):
    if s not in Marq:
        Marq.append(s) #Si le sommet n'est pas encore marqué on le marque en le mettant dans un tableau

    sommets_voisins_non_visites = [] #Création d'une table pour les voisins non visités
    
    for v in graphe[s]: #Pour toutes les valeurs du sommet s
        
        if v not in Marq: #Si la valeur n'est pas marquée on l'ajoute dans la table des voisins non visités
            
            sommets_voisins_non_visites.append(v)
    for sommet in sommets_voisins_non_visites: #On lance le parcours en profondeur pour tout les sommets dans la liste des voisins non visités
        
        parcours_en_profondeur(graphe, sommet, Marq)


Maintenant pour savoir si le graphe est connexe on regarde si on a parcouru tous les sommets du graphe

In [ ]:
def connexe(graphe, Marq):

    if len(Marq) == len(graphe): #renvoie True si la longueur des sommets marqués est égale au nombre de sommets
        return True
    else:
        return False

In [ ]:
def est_connexe(G):

    Dic = MatToDico(G) #On met le graphe G en forme de dictionnaire dans la variable Dic
    Marq = [] #Tableau des sommets marqués
    parcours_en_profondeur(Dic,0,Marq) #On lance le parcours en profondeur avec comme point de depart le sommet 0.

    return connexe(Dic,Marq)

print(est_connexe(Mat))

In [ ]:
#---------Test---------
Graphe1 = MatriceDeso(4)
for li in Graphe1:
    print(li)
print("----------------------------------")
print(est_connexe(Graphe1))


#### Circuit Eulerien
##### 1. Existence d'un circuit
Un graphe admet un circuit eulérien si le graphe est connexe et s'il possède `0` sommet à degré impair ou `2` sommets à degrés impairs

In [ ]:

def Possede_circuit_e(G):
    if not est_connexe(G):
        print("Le Graphe n'est pas connexe")
        return False
    imp = 0 #Compteur de sommets à degrés impairs
    
    Dic = MatToDico(G) #On met le graphe G en forme de dictionnaire dans la variable Dic

    for val in MatToDico(G): #On parcours les sommets du graphes
        
        if not len(Dic[val]) % 2 == 0: #Si le degré est impair on incrémente imp de 1
            imp += 1
    
    if imp == 0 or imp == 2: #On renvoie True si le compteur est égal à 0 (que des degrés pairs) ou égal à 2 (2 degrés impairs)
        return True
    return False #Sinon on renvoie False

#---------Test---------
Graphe2 = MatriceDeso(4)
for li in Graphe2:
    print(li)

Possede_circuit_e(Graphe2)
    



##### 2. Algorithme de Hierholzer

Maintenant que nous savons determiner si un graphe est eulerien ou pas, appliquons l'algorithme de Hierholzer.  

l'algorithme de Hierholzer permet de déterminer un chemin eulérien.

Voici les étapes de l'algorithme :
- Vérifier si le graphe admet un chemin eulérien
- Choisir un sommet de départ.
- Avancer jusqu'à retomber sur le sommet de depart en ne repassant pas par le même sommet --> C1 
- Faire de même pour les sommets du C1 s'ils ont des voisins qui n'ont pas été visités ---> C1
- Répetér jusqu'à ce que tous les sommets soient parcourus

In [ ]:
def hier(G): #parcours_en_pronfondeur ressemble à l'algorithme de Hierholzer
    if not Possede_circuit_e(G):
        return False
    Dic = MatToDico(G)
        
    chem = []
    parcours_en_profondeur(MatToDico(G),0,chem)
    print('-------------------')
    print(chem)

In [ ]:
#---------Test---------

Graphe2 = MatriceDeso(6)
while hier(Graphe2) == False: #Lance la fonction tant qu'il n'y a pas un cas qui marche
    Graphe2 = MatriceDeso(6)
    for li in Graphe2:
        print(li)
    print('--------------------------------')
dessineGraphe(Graphe2)
hier(Graphe2)

# III - Tentative Animation

In [ ]:
def racineNieme(n) : 
    res=[]
    for i in range(n) : res.append([cos(2*pi*i/n), sin(2*pi*i/n)]) 
    return res

def dessineGraphe(M, couleur='r') : 
    nb_sommet = len(M)
    
    sommet = racineNieme(nb_sommet)
    for i in range(nb_sommet) : text(sommet[i][0], sommet[i][1], "$x_{"+str(i+1)+"}$")

    for i in range(nb_sommet) :
        for j in range(nb_sommet) : 
            if(M[i][j]!=0) : 
                plot([sommet[i][0], sommet[j][0]], [sommet[i][1], sommet[j][1]], couleur)
    show()

    
G = MatriceDeso(6)
nb_sommet = len(G)
sommet = racineNieme(nb_sommet)
fig = figure(figsize=(5,5), dpi=120, facecolor="lightgrey")


f, = plot([],[],'r')
ylim(-2,2)
xlim(-2,2)



def MonAnimation(num_frame,couleur ='r'):
    nb_sommet = len(G)
    
    sommet = racineNieme(nb_sommet)
    for i in range(nb_sommet) : text(sommet[i][0], sommet[i][1], "$x_{"+str(i+1)+"}$")

    for i in range(nb_sommet) :
        for j in range(nb_sommet) : 
            if(G[i][j]!=0) : 
                f.set_data([sommet[i][0], sommet[j][0]],[sommet[i][1], sommet[j][1]])  
    return()
ani = animation.FuncAnimation(fig,MonAnimation,nb_sommet+1)

HTML(ani.to_jshtml())